## load model

In [1]:
with open("hf.token", "r") as f:
    hftoken = f.read().strip()  

import os
cache_dir = "/mnt/c/Users/yc/.cache/huggingface"
os.environ['HF_HOME'] = cache_dir

from huggingface_hub import login
login(token=hftoken)  # Move token to environment variable

from ult import *

/home/yc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
device_map = {"": 0}
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards: 100%|██████████| 4/4 [01:22<00:00, 20.66s/it]


## data

In [32]:
def txt_to_dict(file_path):
    data_dict = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines) - 1, 2):
            key = lines[i].strip()    # Odd line are key
            value = lines[i + 1].strip()  # Even line are value
            data_dict[key] = value

    return data_dict

txt_file_path = 'data/formaldef.txt'
formaldic = txt_to_dict(txt_file_path)
len(formaldic)

meddict={}
for k,v in formaldic.items():
    meddict[k.split('Listen to pronunciation')[0].split('(')[0]]=v
filename= 'data/filtered_medical_dictionary.csv'
eighth_grade_words=set()
with open(filename, 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        if row:  # Make sure row is not empty
            eighth_grade_words.add(row[0])  # Add the word (first column)
filtered_meddict = {word: explanation for word, explanation in meddict.items() 
                   if word in eighth_grade_words}
meddict=filtered_meddict
# load data
df = pd.read_csv('data/CORAL/coral-expert-curated-medical-oncology-reports-to-advance-language-model-inference-1.0/coral/unannotated/data/breastca_unannotated.csv')
# df = df.sample(1, random_state=42)
df=df.iloc[[83, 53]] 
test_note=df.iloc[0]['note_text']
len(df)

2

# save

In [33]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Dict, Optional, Tuple
import json
import gc

def run_model_with_cache_manual(
    prompt_text: str, 
    model, 
    tokenizer, 
    generation_config: Dict, 
    kv_cache: Optional[Tuple[torch.Tensor]] = None
) -> Tuple[str, Tuple[torch.Tensor]]:
    """
    Memory-efficient manual generation with proper cleanup.
    """
    
    # 1. Tokenize the new input text
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    input_ids = inputs["input_ids"]
    
    # 2. Setup generation parameters
    max_new_tokens = generation_config.get("max_new_tokens", 256)
    eos_token_id = generation_config.get("eos_token_id", tokenizer.eos_token_id)
    do_sample = generation_config.get("do_sample", False)
    
    # 3. Determine the starting position and build attention mask
    if kv_cache is not None:
        past_seq_len = kv_cache[0][0].shape[2]
        
        # Build full attention mask ONCE upfront
        attention_mask_cached = torch.ones((1, past_seq_len), dtype=torch.long, device=model.device)
        attention_mask_new = inputs.get("attention_mask", torch.ones_like(input_ids))
        attention_mask = torch.cat([attention_mask_cached, attention_mask_new], dim=1)
    else:
        attention_mask = inputs.get("attention_mask", torch.ones_like(input_ids))
    
    # 4. First forward pass to process the new prompt
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            past_key_values=kv_cache,
            use_cache=True
        )
    
    current_cache = outputs.past_key_values
    next_token_logits = outputs.logits[:, -1, :]
    
    # Clear outputs to free memory
    del outputs
    
    # 5. Pre-allocate list for generated tokens
    generated_tokens = []
    
    # 6. Generation loop
    for step in range(max_new_tokens):
        # Sample next token
        if do_sample:
            temperature = generation_config.get("temperature", 1.0)
            probs = torch.softmax(next_token_logits / temperature, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
        else:
            # Greedy decoding
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
        
        # Store the token value
        token_id = next_token.item()
        generated_tokens.append(token_id)
        
        # Check for EOS
        if token_id == eos_token_id:
            break
        
        # Prepare for next iteration - extend attention mask efficiently
        # Instead of concatenating, create a new mask of the right size
        new_length = attention_mask.shape[1] + 1
        new_attention_mask = torch.ones((1, new_length), dtype=torch.long, device=model.device)
        
        # Free old attention mask
        del attention_mask
        attention_mask = new_attention_mask
        
        # Forward pass with the new token
        with torch.no_grad():
            outputs = model(
                input_ids=next_token,
                attention_mask=attention_mask,
                past_key_values=current_cache,
                use_cache=True
            )
        
        # Update cache and logits
        current_cache = outputs.past_key_values
        next_token_logits = outputs.logits[:, -1, :]
        
        # Clean up
        del outputs, next_token
    
    # 7. Decode the generated tokens
    raw_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    # 8. Cleanup
    del attention_mask, next_token_logits, generated_tokens
    
    return raw_output.strip(), current_cache


# --- EXAMPLE USAGE WITH MEMORY MANAGEMENT ---
if __name__ == "__main__":
    
    # Shared generation config - REDUCED max_new_tokens to save memory
    gen_config = {
        "max_new_tokens": 256,  # Reduced from 256
        "eos_token_id": tokenizer.eos_token_id,
        "do_sample": False
    }

    myprint("\n--- 1. Creating Base KV Cache from long text... ---")
    
    base_prompt = (
        f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
        f"You are a medical data extraction expert. You will be given a long medical note. "
        f"Your task is to answer a series of questions about it, one by one. "
        f"Respond *only* with the extracted data in JSON format."
        f"<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
        f"Here is the medical note:\n\n"
        f"--- BEGIN NOTE ---\n{test_note}\n--- END NOTE ---"
        f"\n\nI will now ask you to extract specific sections. "
        f"Please wait for my first extraction task."
        f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        f"{{\"status\": \"Understood. I have read the note and am ready.\"}}"
    )
    
    myprint("  Tokenizing and running forward pass to get base cache...")
    with torch.no_grad():
        inputs = tokenizer(base_prompt, return_tensors="pt").to(model.device)
        
        outputs = model(
            input_ids=inputs["input_ids"], 
            attention_mask=inputs["attention_mask"],
            use_cache=True
        )
        
        base_cache = outputs.past_key_values
        
        # Clean up
        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()
    
    myprint("Base KV Cache created. Ready for 'branching' extractions.")

    # --- 2. Define the "Branching" Extraction Tasks ---
    extraction_prompts = {
        "Reason_for_Visit": "Extract 'Reason for Visit' (e.g., new patient, chemo, toxicity, surveillance, lab review, etc.).",
        "What_We_Found": "Extract 'What We Found' (e.g., summary of imaging, key labs, performance status, new findings).",
        "Treatment_Summary": "Extract 'Treatment Summary' (e.g., current medications, recent changes, supportive meds, bowel regimen, blood transfusion).",
        "What_We_Discussed": "Extract 'What We Discussed / Decided' (e.g., goals of treatment, response, side effects, clinical trials).",
        "medication_plan": "Extract future medication plan (including changes to current medications, supportive meds, bowel regimen, blood transfusion)",
        "imaging_plan": "Extract future imaging plan (such as CT/MRl/PET/ultrasound/ DEXA scan)",
        "lab_plan": "Extract future lab plan (such as CBC, CMP, tumor markers, coagulation profile, etc. specify frequency and rationale)",
        "genetic_testing_plan": "Extract future molecular or genetic testing plan (tumor sequencing, germline panel, liquid biopsy)",
        "procedure_plan": "Extract future procedure plan (such as Surgery, Radiation therapy, interventional procedures)",
        "Admin_Notes": "Extract 'Administrative / Practical Notes' (e.g., refills, code status)."
    }

    extracted_data = {}

    myprint("\n--- 2. Running EFFICIENT 'Branching' Extractions ---")

    for key, task in extraction_prompts.items():
        myprint(f"\nExtracting: {key}...")
        
        task_prompt = (
            f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
            f"TASK: {task}\n\nRespond *only* with a JSON object."
            f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        )

        # Use the SAME base_cache for each extraction
        answer, returned_cache = run_model_with_cache_manual(
            task_prompt, 
            model, 
            tokenizer, 
            gen_config, 
            kv_cache=base_cache
        )
        
        # CRITICAL: Explicitly delete the returned cache to free memory
        del returned_cache
        
        # Force garbage collection and clear GPU cache
        torch.cuda.empty_cache()
        gc.collect()
        
        myprint(f"  Raw Output: {answer}")
        
        try:
            clean_answer = answer.strip().strip("```json").strip("```").strip()
            extracted_data[key] = json.loads(clean_answer)
        except json.JSONDecodeError:
            extracted_data[key] = {"error": "Failed to parse JSON", "raw": answer}

    myprint("\n--- 3. All extractions complete. ---")
    myprint("\n--- FINAL EXTRACTED DATA ---")
    myprint(json.dumps(extracted_data, indent=2))

 --- 1. Creating Base KV Cache from long text... ---
  Tokenizing and running forward pass to get base cache...
Base KV Cache created. Ready for 'branching' extractions.
 --- 2. Running EFFICIENT 'Branching' Extractions ---
 Extracting: Reason_for_Visit...
  Raw Output: {   "Reason for Visit": "Follow-up" }
 Extracting: What_We_Found...
  Raw Output: {   "Imaging": {     "Diagnostic mammogram": "irregular mass (6.7 x 5.2 x 3.7 cm) in upper outer quadrant of left breast,
middle to posterior third",     "Left breast ultrasound": "multiple irregular masses with probable ductal extension between them",     "Left
axillary ultrasound": "at least 5 abnormally thickened lymph nodes",     "CT CAP": "spiculated left breast mass (5 x 3 cm); multiple left
axillary LN (ex - 1.2 x 1.0 cm); no evidence of distant metastases, but subtle sclerotic lesion of left posterior 7th rib",     "Bone scan":
"non-specific small mild focal uptake of posterior left 7th rib",     "PET/CT": "suspicious FDG-avid lyti

In [45]:
res=[]

for row in df.itertuples():
    test_note=row.note_text
    gen_config = {
    "max_new_tokens": 512,  # <-- Increased from 150 to 512
    "eos_token_id": tokenizer.eos_token_id,
    "do_sample": False
    }

    # myprint("\n--- 1. Creating Base KV Cache from long text... ---")

    base_prompt = (
        f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
        f"You are a medical data extraction expert. You will be given a long medical note. "
        f"Your task is to answer a series of questions about it, one by one. "
        # FIX 2: Stricter instruction in system prompt
        f"Respond *only* with the valid JSON object requested. Do not add markdown backticks or any other text."
        f"<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
        f"Here is the medical note:\n\n"
        f"--- BEGIN NOTE ---\n{test_note}\n--- END NOTE ---"
        f"\n\nI will now ask you to extract specific sections. "
        f"Please wait for my first extraction task."
        f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        f"{{\"status\": \"Understood. I have read the note and am ready.\"}}"
    )

    # myprint("  Tokenizing and running forward pass to get base cache...")
    with torch.no_grad():
        inputs = tokenizer(base_prompt, return_tensors="pt").to(model.device)
        
        # This logic to create the base_cache is correct
        outputs = model(
            input_ids=inputs["input_ids"], 
            attention_mask=inputs["attention_mask"],
            use_cache=True
        )
        
        base_cache = outputs.past_key_values
        
        # Clean up
        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()

    # myprint("Base KV Cache created. Ready for 'branching' extractions.")

    # --- FIX 3: Define STRICT SCHEMAS for each extraction task ---
    # This is the most important "prompt engineering" fix.
    # We are telling the model *exactly* what keys to use.
    extraction_prompts = {
        "Reason_for_Visit": """
    TASK: Extract 'Reason for Visit'.
    Respond *only* with a JSON object using this exact schema:
    {
    "visit_type": "e.g., New patient, Surveillance, Side effect management",
    "summary": "A brief summary of the reason for visit."
    }
    """,
        "What_We_Found": """
    TASK: Extract 'What We Found'.
    Respond *only* with a JSON object using this exact schema:
    {
    "imaging_summary": "Summary of most recent imaging.",
    "lab_summary": "Summary of key lab results.",
    "performance_status": "e.g., ECOG 0, Stable",
    "findings": "Summary of new findings or disease status."
    }
    """,
        "Treatment_Summary": """
    TASK: Extract 'Treatment Summary'.
    Respond *only* with a JSON object using this exact schema:
    {
    "current_meds": "List of current oncologic medications or regimens.",
    "recent_changes": "Any holds, dose reductions, or switches.",
    "supportive_meds": "List of supportive medications."
    }
    """,
        "What_We_Discussed": """
    TASK: Extract 'What We Discussed / Decided'.
    Respond *only* with a JSON object using this exact schema:
    {
    "goals_of_treatment": "e.g., Curative, Palliative, Surveillance",
    "response_assessment": "How the cancer is responding.",
    "side_effects_discussed": "Which side effects were discussed.",
    "other_discussions": "e.g., Clinical trials, palliative care"
    }
    """,
        "Plan_Going_Forward": """
    TASK: Extract 'Plan Going Forward'.
    Respond *only* with a JSON object using this exact schema:
    {
    "medication_plan": "e.g., Continue, Start, Stop",
    "imaging_plan": "Which scans and when.",
    "lab_plan": "Which labs and when.",
    "referrals": "List of any new referrals.",
    "follow_up": "When the next visit is."
    }
    """,
        "Admin_Notes": """
    TASK: Extract 'Administrative / Practical Notes'.
    Respond *only* with a JSON object using this exact schema:
    {
    "refills": "Any medications refilled.",
    "code_status": "e.g., Full code, DNR"
    }
    """
    }

    extracted_data = {}

    # myprint("\n--- 2. Running EFFICIENT 'Branching' Extractions ---")

    # (Assuming `run_model_with_cache_manual` is your function `run_model_with_cache`)
    # If not, please rename this call to `run_model_with_cache`
    run_model_function = run_model_with_cache_manual 

    for key, task in extraction_prompts.items():
        # myprint(f"\nExtracting: {key}...")
        
        # --- FIX 4: Remove the BOS token from the loop ---
        # The base_cache *already* contains the BOS token.
        # Adding it again can cause errors.
        task_prompt = (
            f"<|start_header_id|>user<|end_header_id|>\n\n" # <-- Removed <|begin_of_text|>
            f"{task}" # <-- Use the new, detailed prompt
            f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        )

        # Use the SAME base_cache for each extraction
        answer, returned_cache = run_model_function(
            task_prompt, 
            model, 
            tokenizer, 
            gen_config, 
            kv_cache=base_cache
        )
        
        # Your memory management here is good.
        del returned_cache
        torch.cuda.empty_cache()
        gc.collect()
        
        # myprint(f"  Raw Output: {answer}")
        
        try:
            # We are now *only* expecting a JSON object
            clean_answer = answer.strip().strip("```json").strip("```").strip()
            extracted_data[key] = json.loads(clean_answer)
        except json.JSONDecodeError:
            extracted_data[key] = {"error": "Failed to parse JSON", "raw": answer}

    # myprint("\n--- 3. All extractions complete. ---")
    # myprint("\n--- FINAL EXTRACTED DATA ---")
    res.append(json.dumps(extracted_data, indent=2))



In [46]:
resdf=[json.loads(r) for r in res]
for i in range(2):
    test_note=df.iloc[i]['note_text']
    resdf[i]['note']=test_note

In [47]:
resdf=pd.DataFrame(resdf)
resdf.to_csv("keysummaryv1.csv", index=False)

In [48]:
resdf

,Reason_for_Visit,What_We_Found,Treatment_Summary,What_We_Discussed,Plan_Going_Forward,Admin_Notes,note
0,{'visit_type': 'Ongoing management and treatme...,{'imaging_summary': 'Extensive osseous metasta...,{'current_meds': 'capecitabine (XELODA) 1500 m...,"{'goals_of_treatment': 'Palliative', 'response...",{'medication_plan': 'Continue xeloda 1500mg BI...,"{'refills': '', 'code_status': ''}",We performed this consultation using real-time...
1,"{'visit_type': 'Follow-up', 'summary': 'The pa...",{'imaging_summary': 'PET/CT: significant impro...,"{'current_meds': 'leuprolide, letrozole, zoled...","{'goals_of_treatment': 'Prolongation of life, ...",{'medication_plan': 'Continue leuprolide every...,{'refills': 'calcium citrate/vitamin D3 (CITRA...,ID: ***** ***** is a 39 y.o. premenopausal pat...
